# Create Nanoribbon Edge Passivation

Passivate nanoribbon edges by detecting and adding passivants to undercoordinated atoms (atoms with fewer neighbors than bulk).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (nanoribbon) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete.
1. Scroll down to view results.

## Notes

1. Coordination-based passivation finds atoms with fewer neighbors than expected
2. Coordination threshold below which passivation is applied, must be set depending on the material
3. Passivant atoms are placed along the direction away from existing bonds


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior


In [ ]:
# Material selection
MATERIAL_INDEX = 16  # Which material to use from input list

# Passivation parameters
PASSIVANT = "H"  # Chemical symbol of passivating atom
BOND_LENGTH = 1.0  # Distance from atom to passivant (Å)
COORDINATION_THRESHOLD = 6  # Minimum neighbors before passivation needed

# Neighbor detection parameters
NEIGHBOR_RADIUS = 2.5  # Distance to look for neighbors (Å)
SEARCH_DEPTH = 5.0  # How deep to look for surface atoms (Å)

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [1, 1, 1]  # Structure repeat in view

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Load and preview input material

In [ ]:
from utils.jupyterlite import get_materials
from utils.visualize import visualize_materials

materials = get_materials(globals())
nanoribbon = materials[MATERIAL_INDEX]

if SHOW_INTERMEDIATE_STEPS:
    print("Initial nanoribbon structure:")
    visualize_materials(nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

## 2. Analyze Coordination Numbers

In [ ]:
from mat3ra.made.tools.build.passivation import get_unique_coordination_numbers,PassivationConfiguration

config = PassivationConfiguration(
    slab=nanoribbon,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH
)
coordination_numbers = get_unique_coordination_numbers(config)
print("\nCoordination numbers:", coordination_numbers)

## 3. Configure Passivation
### 3.1. Set up passivation configuration and builder


In [ ]:
from mat3ra.made.tools.build.passivation.builders import CoordinationBasedPassivationBuilder, CoordinationBasedPassivationBuilderParameters

builder_params = CoordinationBasedPassivationBuilderParameters(
    shadowing_radius=NEIGHBOR_RADIUS,
    depth=SEARCH_DEPTH,
    coordination_threshold=COORDINATION_THRESHOLD
)

builder = CoordinationBasedPassivationBuilder(build_parameters=builder_params)

### 3.2. Generate passivated structure


In [ ]:
from mat3ra.made.tools.build.passivation import create_passivation

passivated_nanoribbon = create_passivation(config, builder)

## 4. Preview the passivated structure

In [ ]:
print("Final passivated structure:")
visualize_materials(passivated_nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
visualize_materials(passivated_nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(passivated_nanoribbon)